### 네이버 영화 리뷰 감성분석 2
- tokenizer 함수
- TfidfVectorizer + NaiveBayes

In [1]:
import numpy as np
import pandas as pd
import warnings
warnings.filterwarnings('ignore')

In [2]:
train_df = pd.read_csv('data/네이버영화리뷰_train_전처리완료.tsv', sep='\t')
test_df = pd.read_csv('data/네이버영화리뷰_test_전처리완료.tsv', sep='\t')

In [7]:
X_train = train_df.document.values
X_test = test_df.document.values
y_train = train_df.label.values
y_test = test_df.label.values

##### 1. Tokenizer 함수

In [3]:
with open('data/불용어.txt') as st:
    lines = st.readlines()
stop_words = [line.split('\t')[0] for line in lines]

In [4]:
from konlpy.tag import Okt
okt = Okt()

In [5]:
def okt_tokenizer(text):
    morphs = okt.morphs(text, stem=True)
    tokens = [word for word in morphs if word not in stop_words]
    return tokens

##### 2. Pipeline 학습

In [6]:
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB

In [8]:
pipeline = Pipeline([
    ('TVECT', TfidfVectorizer(tokenizer=okt_tokenizer)),
    ('NB', MultinomialNB())
])
%time pipeline.fit(X_train, y_train)

CPU times: total: 5min 33s
Wall time: 5min 21s


Pipeline(steps=[('TVECT',
                 TfidfVectorizer(tokenizer=<function okt_tokenizer at 0x000001BF5BEEB490>)),
                ('NB', MultinomialNB())])

In [9]:
%time pipeline.score(X_test, y_test)

CPU times: total: 1min 42s
Wall time: 1min 42s


0.8356464423155654

##### 3. 실제 데이터 적용

In [18]:
reviews = [
    '이 영화 개꿀잼 ㅋㅋㅋ',
    '이 영화 핵노잼 ㅠㅠ',
    '이딴게 영화냐 ㅉㅉ',
    '감독 뭐하는 놈이냐?',
    '와 개쩐다 정말 세계관 최강자들의 영화다',
    '재미있으면서 재미없다',
    '재미 없는듯 재미있다'
]

In [19]:
import re
reviews = list(map(lambda s: re.sub('[^ㄱ-ㅎㅏ-ㅣ가-힣]', ' ', s), reviews))
reviews

['이 영화 개꿀잼 ㅋㅋㅋ',
 '이 영화 핵노잼 ㅠㅠ',
 '이딴게 영화냐 ㅉㅉ',
 '감독 뭐하는 놈이냐 ',
 '와 개쩐다 정말 세계관 최강자들의 영화다',
 '재미있으면서 재미없다',
 '재미 없는듯 재미있다']

In [20]:
pipeline.predict(reviews)

array([1, 0, 0, 0, 1, 0, 1], dtype=int64)

In [21]:
pipeline.predict_proba(reviews)

array([[0.09911364, 0.90088636],
       [0.92294195, 0.07705805],
       [0.97925611, 0.02074389],
       [0.92628194, 0.07371806],
       [0.34683511, 0.65316489],
       [0.64282535, 0.35717465],
       [0.36231249, 0.63768751]])

##### 최적 파라메터 찾기
- 매 시행마다 한글 형태소 분석을 하느라 시간이 많이 소요됨
- 최적 파라메터를 찾으려면 형태소 분석이 끝난 데이터로 할 것

In [23]:
# CountVectorizer에서 찾은 최적의 파라메터를 TfidfVectorizer에 적용
pipeline2 = Pipeline([
    ('TVECT', TfidfVectorizer(tokenizer=okt_tokenizer, ngram_range=(1,2), max_df=0.95)),
    ('NB', MultinomialNB())
])
%time pipeline2.fit(X_train, y_train)

CPU times: total: 6min
Wall time: 5min 57s


Pipeline(steps=[('TVECT',
                 TfidfVectorizer(max_df=0.95, ngram_range=(1, 2),
                                 tokenizer=<function okt_tokenizer at 0x000001BF5BEEB490>)),
                ('NB', MultinomialNB())])

In [24]:
%time pipeline2.score(X_test, y_test)

CPU times: total: 1min 48s
Wall time: 1min 49s


0.8609882911651519